In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from dnn_tau import Dnn_tau
from data_extractor import Data_extractor_v4, output_vars_v4
import os
import fnmatch
from utils import normalize, bucketize, split_dataset, flatten_2D_list
from copy import deepcopy
import pickle

In [2]:
path = "/data/hnl/prompt_tau/anatuple/nanoV10/TEST10/"
features = deepcopy(output_vars_v4)
features.extend(['signal_label', 'channel', 'event_type', 'mass_hyp'])
# channels = os.listdir(path)
channels = ['tee', 'tem', 'tmm', 'tte', 'ttm']
relative_path = "/anatuple/"

In [3]:
values = []
flat_features = flatten_2D_list(features)
for i in range(len(flat_features)):
    values.append([])
data = dict(zip(flat_features, values))

for channel in channels:
    extractor = Data_extractor_v4(channel)
    data = extractor(path+channel+relative_path, data=data)

In [4]:
data_dict = data

In [5]:
data = pd.DataFrame(data_dict)
data = data.rename(columns={"genWeight": "weightOriginal"})
weightNorm = deepcopy(data['weightOriginal'])
data['weightNorm'] = weightNorm

This script seems to be doing a custom extraction of specific data from a large dataset stored in a directory tree, which may be organized by categories or "channels" like 'tee', 'tem', 'tmm', 'tte', and 'ttm'. It appears to be extracting information related to particle physics research, particularly related to lepton (a type of subatomic particle) interactions. Here is a breakdown of what it does:

1. **Initialization**: The script starts by defining a path to the dataset. It also creates a list of features (variables) it intends to extract from the dataset. The features include a base list defined by the variable `output_vars_v4`, and other added features like 'signal_label', 'channel', 'event_type', and 'mass_hyp'. It then initializes a Python dictionary `data` with keys being the names of the features and values being empty lists. 

2. **Data Extraction**: It then loops over a predefined list of channels ('tee', 'tem', 'tmm', 'tte', and 'ttm'). For each channel, it creates an instance of a class `Data_extractor_v4` with the channel as an argument, and calls its method with the path to the data of the channel and the dictionary `data`. 

The class `Data_extractor_v4` is a subclass of `Data_extractor` and is initialized with the specific channel. It defines multiple attributes like `raw_vars_general`, `raw_vars_lepton1`, `raw_vars_lepton2`, `raw_vars_lepton3`, `output_vars`, `functions`, and `input_vars`. These attributes likely determine the extraction process. The extraction method of this class seems to extract specific variables from the data files in a given directory (e.g., '/data/hnl/prompt_tau/anatuple/nanoV10/TEST10/tee') and update the dictionary `data` with the extracted information.

3. **Storage**: After all channels have been processed, the script should have a dictionary `data` with keys being the names of the features and values being lists filled with the corresponding data from the dataset.

4. **Functions**: The script also references a number of functions (`deltaphi`, `deltaeta`, `deltaR`, `sum_pt`, `transverse_mass`, `invariant_mass`, `total_transverse_mass`, `HNL_CM_angles_with_MET`, `W_CM_angles_to_plane`, `W_CM_angles_to_plane_with_MET`, `HNL_CM_masses`, `HNL_CM_masses_with_MET`, `W_CM_angles`, `count_tauh`). These likely represent various calculations relevant to the particle physics research, such as calculating angular differences, invariant mass, transverse mass, etc.

It's important to note that the actual function of the script depends on the specifics of the `Data_extractor_v4` class and the underlying `Data_extractor` class, which aren't provided. So this analysis is somewhat speculative, based on the names and structures of the objects and methods used.


In [6]:
N = len(data['event'])
data_norm = normalize(pd.DataFrame(data), 'mass_hyp', N, weight_name='weightNorm')
data_norm = normalize(data_norm, 'signal_label', N, weight_name='weightNorm')
data_norm = normalize(data_norm, 'channel', N/5, weight_name='weightNorm')
data_processed, channel_indices = bucketize(data_norm, 'channel')
print(list(data_processed.keys()))
print(channel_indices)
print(N)
print(sum(data_processed['weightNorm']))

['event', 'weightOriginal', 'charge_1', 'charge_2', 'charge_3', 'pt_1', 'pt_2', 'pt_3', 'pt_MET', 'eta_1', 'eta_2', 'eta_3', 'mass_1', 'mass_2', 'mass_3', 'deltaphi_12', 'deltaphi_13', 'deltaphi_23', 'deltaphi_1MET', 'deltaphi_2MET', 'deltaphi_3MET', 'deltaphi_1(23)', 'deltaphi_2(13)', 'deltaphi_3(12)', 'deltaphi_MET(12)', 'deltaphi_MET(13)', 'deltaphi_MET(23)', 'deltaphi_1(2MET)', 'deltaphi_1(3MET)', 'deltaphi_2(1MET)', 'deltaphi_2(3MET)', 'deltaphi_3(1MET)', 'deltaphi_3(2MET)', 'deltaeta_12', 'deltaeta_13', 'deltaeta_23', 'deltaeta_1(23)', 'deltaeta_2(13)', 'deltaeta_3(12)', 'deltaR_12', 'deltaR_13', 'deltaR_23', 'deltaR_1(23)', 'deltaR_2(13)', 'deltaR_3(12)', 'pt_123', 'mt_12', 'mt_13', 'mt_23', 'mt_1MET', 'mt_2MET', 'mt_3MET', 'mt_1(23)', 'mt_2(13)', 'mt_3(12)', 'mt_MET(12)', 'mt_MET(13)', 'mt_MET(23)', 'mt_1(2MET)', 'mt_1(3MET)', 'mt_2(1MET)', 'mt_2(3MET)', 'mt_3(1MET)', 'mt_3(2MET)', 'mass_12', 'mass_13', 'mass_23', 'mass_123', 'Mt_tot', 'HNL_CM_angle_with_MET_1', 'HNL_CM_angle_w

In [19]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)

# Create the directory if it doesn't already exist
output_dir = os.path.join(parent_dir, "extracted_data")
os.makedirs(output_dir, exist_ok=True)

output_file_path = os.path.join(output_dir, "TEST10_channel_indices")

with open(output_file_path, 'wb') as file:
    pickle.dump(channel_indices, file)

# print(channel_indices)

{'tee': 0, 'tem': 1, 'tmm': 2, 'tte': 3, 'ttm': 4}


In [15]:
data_processed.to_pickle(parent_dir+ "/extracted_data/TEST10_v4_total")

In [16]:
selection = ['charge_1', 'charge_2', 'charge_3', 'pt_1',
       'pt_2', 'pt_3', 'pt_MET', 'eta_1', 'eta_2', 'eta_3', 'mass_1', 'mass_2',
       'mass_3', 'deltaphi_12', 'deltaphi_13', 'deltaphi_23', 'deltaphi_1MET',
       'deltaphi_2MET', 'deltaphi_3MET', 'deltaphi_1(23)', 'deltaphi_2(13)',
       'deltaphi_3(12)', 'deltaphi_MET(12)', 'deltaphi_MET(13)',
       'deltaphi_MET(23)', 'deltaphi_1(2MET)', 'deltaphi_1(3MET)',
       'deltaphi_2(1MET)', 'deltaphi_2(3MET)', 'deltaphi_3(1MET)',
       'deltaphi_3(2MET)', 'deltaeta_12', 'deltaeta_13', 'deltaeta_23',
       'deltaeta_1(23)', 'deltaeta_2(13)', 'deltaeta_3(12)', 'deltaR_12',
       'deltaR_13', 'deltaR_23', 'deltaR_1(23)', 'deltaR_2(13)',
       'deltaR_3(12)', 'pt_123', 'mt_12', 'mt_13', 'mt_23', 'mt_1MET',
       'mt_2MET', 'mt_3MET', 'mt_1(23)', 'mt_2(13)', 'mt_3(12)', 'mt_MET(12)',
       'mt_MET(13)', 'mt_MET(23)', 'mt_1(2MET)', 'mt_1(3MET)', 'mt_2(1MET)',
       'mt_2(3MET)', 'mt_3(1MET)', 'mt_3(2MET)', 'mass_12', 'mass_13',
       'mass_23', 'mass_123', 'Mt_tot', 'HNL_CM_angle_with_MET_1',
       'HNL_CM_angle_with_MET_2', 'W_CM_angle_to_plane_1',
       'W_CM_angle_to_plane_2', 'W_CM_angle_to_plane_with_MET_1',
       'W_CM_angle_to_plane_with_MET_2', 'HNL_CM_mass_1', 'HNL_CM_mass_2',
       'HNL_CM_mass_with_MET_1', 'HNL_CM_mass_with_MET_2', 'W_CM_angle_12',
       'W_CM_angle_13', 'W_CM_angle_23', 'W_CM_angle_1MET', 'W_CM_angle_2MET',
       'W_CM_angle_3MET', 'mass_hyp', 'signal_label']

In [18]:
train, val, test, meas = split_dataset(data_processed)
pd.to_pickle(train, parent_dir + "/extracted_data/TEST10_v4_train")
pd.to_pickle(val, parent_dir +"/extracted_data/TEST10_v4_val")
pd.to_pickle(test,parent_dir +  "/extracted_data/TEST10_v4_test")
pd.to_pickle(meas, parent_dir +"/extracted_data/TEST10_v4_meas")

Total number of events :  1798683
Train set : 37.52 %
Validation set : 12.51 %
Test set : 24.99 %
Measurement set : 24.98 %
